# TF-CNN

对[CIFAR-10](http://www.cs.toronto.edu/~kriz/cifar.html)数据集的分类是机器学习中一个公开的基准测试问题，其任务是对一组32*32的RGB图片进行分类，共有十个类别。

    飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

![](./图表/cifar-samples.png)

可以参考[Alex Krizhevsky](http://www.cs.toronto.edu/~kriz/index.html)写的[技术报告](http://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf)。

## 目标

本文旨在建立一个用于识别图片的、相对较小的卷积神经网络，在这一过程中：

- 着重建立一个规范的卷积神经网络结构并进行训练和评估
- 为建立更大规模更加复杂的模型提供一个范例

选择CIFAR-10是因为它的复杂程度足以用来检验TF中的大部分功能，并可将其扩展为更大的模型。与此同时，由于模型较小所以训练速度很快，比较适合用来测试新的想法、检验新的技术。

## 重点

演示了在TF上构建更大更复杂模型的几个要素：

- 相关核心数学对象：[卷积](./卷积.ipynb)、[修正线性激活](./ReLU.ipynb)、[最大池化](./max-pooling.ipynb)以及[局部响应归一化](./局部响应归一化.ipynb)。
- 训练过程网络行为的可视化：输入图片、损失情况、网络行为的分布及梯度。
- 算法学习参数的[移动平均值](http://wiki.mbalib.com/wiki/移动平均法)的计算函数、在评估阶段应用这些平均值提高预测性能。
- 实现一种机制，让学习率随时间推移而递减。
- 为输入数据设计存取[队列](./TF的队列.ipynb)，将磁盘延迟和高开销的图片与处理操作与模型分离处理。

同时提供了多GPU版本：

- 可以配置模型在多个GPU上并行训练
- 可以在多个GPU之间共享和更新张量

本文希望给读者起个好头，在TF上可以为机器视觉方面建立更大型的CNN模型。

## 模型架构

本文所述的模型是一个多层的，由卷积层和非线性层交替多次排列构成。这些层最终通过全连接层对接到Softmax分类器上。这一模型除了最顶部的几层外，基本跟[Alex Krizhevsky](https://code.google.com/p/cuda-convnet/)提出的模型一致。

在一个GPU上经过几个小时的训练，该模型最高可以达到0.86的准确率。模型中包含1068298个学习参数，对一幅图片进行分类大概需要一千九百五十万个乘加操作。

## 代码组织

- cifar10_input.py：读取本地CIFAR-10的二进制文件
- cifar10.py：建立CIFAR-10的模型
- cifar10_train.py：在CPU或GPU上训练CIFAR-10的模型
- cifar10_multi_gpu_train.py：在多GPU上训练CIFAR-10的模型
- cifar10_eval.py：评估CIFAR-10模型的预测性能

### CIFAR-10模型

代码位于cifar10.py，完整的训练图中包含约765个操作。发现通过下面的模块来构造计算图可以最大限度提高代码复用率。

- 输入模型：包括inputs()、distorted_inputs()等一些操作，分别用于读取CIFAR-10图片并做预处理，作为后续评估和训练的输入
- 模型预测：包括inference()等一些操作，用于统计计算，比如给提供的图片进行分类、添加在给定的图片上执行推理的操作（如分类等）
- 模型训练：包括loss()和train()等一些操作，用于计算损失（成本）、计算梯度、进行张量更新以及呈现最终结果

### 模型输入

输入模型是通过inputs()和distorted_inputs()函数建立起来的，这两个函数会从CIFAR-10二进制文件中读取图片信息，由于每个图片的存储字节数目是固定的，因此可以使用tf.FixedLengthRecordReader函数。

图片文件的处理流程如下：

- 图片会被同意裁剪到24x24像素大小，裁剪中央区域用于评估、随机裁剪用于训练
- 图片会进行近似的[白化](./白化.ipynb)处理，使得模型对图片的动态范围变化不敏感

对于训练，另外采取了一系列随机变换的方法来人为地增加数据集的大小：

- 随机左右翻转图片
- 随机变换图片亮度
- 随机改图片对比度

可以在Images（API-Guides-Python-Image）页的列表查看所有可用的变换，对于每张原始图片还附带了一个image_summary以便于在TensorBoard中查看。这对检查输入图片是否正确十分有用。

![](./图表/cifar-image-summary.png)

从磁盘上加载图片并进行变换需要耗费不少时间，为了避免因此导致的训练缓慢，在16个独立的线程中并行操作，这16个线程被连续地安排在一个TensorFlow队列中。

### 模型预测

模型的预测流程有inference()函数构造，该函数会添加必要的操作步骤用于计算预测值的Logits，其对应的模型组织方式如下：

- conv1
  - 实现卷积以及ReLU激活
- pool1
  - 最大池化
- norm1
  - 局部响应归一化
- conv2
  - 卷积和ReLU激活
- norm2
  - 局部响应归一化
- pool2
  - 最大池化
- local3
  - 基于ReLU激活的全连接层
- local4
  - 基于ReLU激活的全连接层
- softmax_linear
  - 做线性变换以输出Logits

这里有个TensorBoard绘制的计算图描述该模型建立的过程：

![](./图表/cifar-graph.png)

#### 练习

- inference的输出是未归一化的Logits，尝试用tf.softmax()修改网络架构后返回归一化的预测值。
- inference的模型跟[cuda-convnet](https://code.google.com/archive/p/cuda-convnet/)所描述的CIFAR-10模型有些许不同，差异主要在于其顶层不是全连接层而是局部连接层，可以尝试修改网络架构来准确的复制全连接模型。

inputs()和inference()提供了评估模型所需的全部构件，现在要讲解的重点是从构件一个模型转向训练一个模型。

### 模型训练

训练一个可进行N维分类的网络的常用方法是[多项逻辑回归](./多项逻辑回归.ipynb)，又叫做Softmax回归。

Softmax回归在网络的输出层附加了一个softmax非线性特征，并且计算归一化的预测值和标签的One-Hot编码的交叉熵。在**[正则化](https://www.cnblogs.com/hanhanzhu000/p/4159833.html "正则化和归一化")**的过程中，会对所有学习变量应用[权重衰减损失](https://www.cnblogs.com/gongxijun/p/8039262.html)。模型的目标函数时求交叉熵损失及所有权重衰减项的和，loss()函数返回的就是这个。

在TensorBoard中使用scalar_summary来查看该值的变化情况：

![](./图表/cifar-loss.png)

使用标准的梯度下降算法来训练模型，其学习率随时间以指数形式衰减：

![](./图表/cifar-lr-decay.png)

train()函数会添加一些操作使得目标函数最小化，这些操作包括计算梯度、更新学习变量。

train()函数最终会返回一个用来对一批图片执行所有计算的操作步骤，以便训练和更新模型。

#### 执行并训练模型

至此已经把模型建立好啦，现在通过执行脚本cifar10_train.py来启动训练过程。

注意：当第一次在CIFAR-10上启动任何任务时，都会自动下载CIFAR-10数据集，大约160M大小，可以看到类似输出：

    Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
    2015-11-04 11:45:45.927302: step 0, loss = 4.68 (2.0 examples/sec; 64.221 sec/batch)
    2015-11-04 11:45:49.133065: step 10, loss = 4.66 (533.8 examples/sec; 0.240 sec/batch)
    2015-11-04 11:45:51.397710: step 20, loss = 4.64 (597.4 examples/sec; 0.214 sec/batch)
    2015-11-04 11:45:54.446850: step 30, loss = 4.62 (391.0 examples/sec; 0.327 sec/batch)
    2015-11-04 11:45:57.152676: step 40, loss = 4.61 (430.2 examples/sec; 0.298 sec/batch)
    2015-11-04 11:46:00.437717: step 50, loss = 4.59 (406.4 examples/sec; 0.315 sec/batch)
    ...

脚本会在每10步训练迭代后打印总损失值以及最后一批数据的处理速度。几点注解：

- 第一批数据会非常慢（大概要几分钟），因为预处理线程要把两万个待处理的CIFAR-10图片填充到重排队列
- 打印出来的损失值是最近一批数据的损失值的均值，请记住损失值是交叉熵和权重衰减项的和
- 上面打印结果中关于一批数据的处理速度实在Tesla K40C上统计出来的（若运行在CPU上性能可能会更低）

#### 练习

- 实验时，第一阶段的训练时间有时会很长，甚至足以令人生厌。可以尝试减少初始化时填充到队列中的图片数量来改善这一情况，在cifar10.py中搜索NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN并修改之。

cifar10_train.py会周期性地在检查点文件中保存模型的所有参数，但不会对模型进行评估。

cifar10_eval.py会使用该检查点文件来测试预测性能。

如果照上述步骤做下来，应该在训练一个CIFAR-10模型的旅程中了。

cifar10_train.py输出的终端信息中提供了关于模型如何训练的一些线索，但是设计者希望了解更多关于模型训练时的细节，比如：

- 损失是真的在减小还是看到的只是噪声数据？
- 为模型提供的图片是否合适？
- 梯度、激活、权重是否合理？
- 目前的学习率是多少？

TensorBoard提供了该功能，可以通过cifar10_train.py的SummaryWriter周期性地获取并显示这些数据。

例如可以在训练过程中查看local3的激活情况及其特征维度的稀疏状况：

![](./图表/cifar-sparsity.png)![](./图表/cifar-activations.png)

**相比总损失，在训练过程中的单项损失尤其值得注意**！

但是由于训练过程中使用的数据批量比较小，损失值中夹杂了相当多的噪声。

实践中，也发现相比于原始值，损失值得移动平均值显得更有意义。参考脚本ExponentialMovingAverage了解如何实现。

### 评估模型

现在可以在另一部分数据集上来评估训练模型的性能。脚本cifar10_eval.py对模型进行了评估，利用inference()函数重构模型，并使用了评估数据集所有（一万张）CIFAR-10图片进行测试。最终计算出准确率为 $\frac{1}{N}$ ，其中N是预测值中指信度最高的一项与图片真实标签匹配的频次。

为了监控模型在训练过程中的改进情况，评估用的脚本文件会周期性地在最新的检查点文件上运行，这些检查点文件是由cifar10_train.py产生。

注意：不要再同一块GPU上同时运行训练程序和评估程序，因为可能会导致显存耗尽，尽可能地在别的单独的GPU上运行评估程序或者在同一块GPU上运行评估程序时暂时挂起训练程序。

可能会看到如下输出：

    2015-11-06 08:30:44.391206: precision @ 1 = 0.860
    ...

评估脚本只是周期性地返回准确率（precision @ 1 = float），该例中返回的准确率是0.86。

cifar10_eval.py同时也返回一些别的可以在TensorBoard中进行可视化的简要信息，可以通过这些内容在评估中进一步了解模型。

训练脚本会为所有学习变量计算其**移动平均值**，评估脚本则直接将所有学习到的模型参数替换成对应的移动平均值。这一替代方法可以在评估过程中提升模型的性能。

#### 练习

通过“precision @ 1 = float”测试发现，使用移动平均值参数可以将预测性能提高约0.03，在cifar10_eval.py中尝试修改不采用移动平均值参数的方式，并确认由此带来的预测性能下降。

### 在多个GPU板卡上训练模型

现代的工作站可能包含多个GPU进行科学计算。TF可以利用这一环境在多个GPU上运行训练程序。

在并行、分布式的环境中进行训练，需要对训练过程进行协调。对于接下来的描述，**术语“模型拷贝”特指“在一个数据子集中训练出来的模型的一份副本”**。

如果天真地对模型参数采用异步方式更新将导致次优的训练性能，这是因为可能会基于一个旧的模型参数的拷贝去训练一个模型。

与此相反，采用完全同步的更新方式，其速度将变得和最慢的模型一样。

在具有多个GPU的工作站中，每个GPU的速度基本接近，并且都含有足够的内存来运行整个CIFAR-10模型。因此选择以下方式来设计训练系统：

- 在每个GPU上放置单独的模型副本
- 等所有GPU处理完一批数据后再同步更新模型的参数

下图示意该模型的结构：

![](./图表/parallelism.png)

可以看到，每个GPU会用一批独立的数据计算梯度和估计。这种设置可以非常有效地将一大批数据分割到各个GPU上。

这一机制要求所有GPU能够共享模型参数。

但是，众所周知，在GPU之间传输数据非常慢！

因此，决定在CPU上存储和更新所有模型的参数（对应上图绿色部分）。

这样一来，GPU在处理一批新的数据之前会更新上一遍学习到的参数。

上图中所有GPU是同步运行的，所有GPU中的梯度会累积并求平均值（绿色）。模型参数会利用所有模型副本梯度的均值来更新。

### 在多个设备中设置变量和操作

在多个设备中设置变量和操作需要做一些特殊的抽象。

首先需要把在单个模型拷贝中计算估计和梯度的行为抽象到一个函数中。代码中，称这个抽象对象为“Tower”。对于每个Tower都需要设置它的两个属性：

- 在一个Tower中为所有操作设置一个唯一的名称：tf.name_scope()通过添加一个范围前缀来提供唯一名称，例如第一个Tower中所有操作都有一个tower_0的前缀（tower_0/conv1/Conv2D）。
- 在一个Tower中优先运行操作的硬件设备：tf.device()提供该信息，例如在第一个Tower中的所有操作都位于“device("/gpu:0")”范围中，暗含的意思是这些操作应该运行在第一块GPU上。

为了在多个GPU上共享变量，所有的变量都绑定在CPU上，并通过tf.get_variable()访问。可以查看[Sharing Variables](./TF共享变量.ipynb)了解如何共享变量。

### 在多个GPU上训练模型启动

如果客户机器上安装多块GPU板卡，可以通过cifar10_multi_gpu_train.py脚本来加速模型的训练。该脚本式训练脚本的一个变种，使用多个GPU实现模型并行训练。

    python cifar10_multi_gpu_train.py --num_gpus=2

该训练脚本输出如下：

    Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
    2015-11-04 11:45:45.927302: step 0, loss = 4.68 (2.0 examples/sec; 64.221 sec/batch)
    2015-11-04 11:45:49.133065: step 10, loss = 4.66 (533.8 examples/sec; 0.240 sec/batch)
    2015-11-04 11:45:51.397710: step 20, loss = 4.64 (597.4 examples/sec; 0.214 sec/batch)
    2015-11-04 11:45:54.446850: step 30, loss = 4.62 (391.0 examples/sec; 0.327 sec/batch)
    2015-11-04 11:45:57.152676: step 40, loss = 4.61 (430.2 examples/sec; 0.298 sec/batch)
    2015-11-04 11:46:00.437717: step 50, loss = 4.59 (406.4 examples/sec; 0.315 sec/batch)
    ...

需要注意的是默认的GPU数量是一，此外，如果机器上只有一块显卡，那么所有计算只会运行在这块显卡上，即便设置的显卡数量是N块！

#### 练习

cifar10_train.py中批处理大小默认是128，尝试在2块GPU上运行cifar10_multi_gpu_train.py脚本，并设置批处理大小为64，然后比较这两种设置的训练速度。

## 下一步

至此已经完成CIFAR-10演练，如果对开发和训练自己的图像分类系统感兴趣，推荐新建一个基于本文脚本的分支，修改其中内容以构建能解决自己问题的图像分类系统。

### 练习

下载[SVHN(Street View House Numbers)](http://ufldl.stanford.edu/housenumbers/)数据集，新建一个CIFAR-10代码分支，将输入数据替换为SVHM的，尝试改变网络结构提高预测性能。